<a href="https://colab.research.google.com/github/searchsolved/search-solved-public-seo/blob/main/search_engine_journal/Top_Traffic_Pages_Search_Console_API_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Find Top Traffic Pages Using Python by LeeFootSEO
##Run All Cells (Runtime > Run All) or press Control + F9
For more of this stuff:

Twitter: https://twitter.com/LeeFootSEO

Patreon (Beta Testing & Early Access): https://www.patreon.com/leefootseo

In [ ]:
!pip install pandas
!pip install git+https://github.com/joshcarty/google-searchconsole
!pip install plotly==5.0.0
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/joshcarty/google-searchconsole to /tmp/pip-req-build-bvzhs1ad
  Running command git clone -q https://github.com/joshcarty/google-searchconsole /tmp/pip-req-build-bvzhs1ad
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import the libraries
import xlsxwriter
import json
import sys
import pandas as pd
import searchconsole
from googleapiclient.discovery import build
from urllib import request
from bs4 import BeautifulSoup
from google.colab import files
import sys
from datetime import date
import datetime
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go
today = datetime.date.today()

In [ ]:
# upload client_id.json
names = files.upload()

In [ ]:
filename=list(names.keys())[0]  # get the name of the uploaded file
account = searchconsole.authenticate(client_config=filename, serialize='credentials.json', flow="console")

IndexError: ignored

# Set GSC Variables Here

In [ ]:
country_filter = "gbr"
url = "https://www.doorsuperstore.co.uk/" # Must match the url displayed in SC exactly. If you're having issues, it's usually missing a training slash!
search_type = "web" # available options are: web, news, video, image, discover & googleNews
start_date = "2022-05-01"  # Example formatting: '2020-09-20'
end_date = "2022-08-01"

# Uncomment to Print the Available Web Properties
Useful if you're having issues matching to a GSC account. Copy the results from here exactly into the URL variable above.

In [ ]:
#print(account.webproperties)

# Pull the Data Using the Search Console API


In [ ]:
dimension_1 = "query"  # get keyword data from Search Console
dimension_2 = "page"  # get URL data from Search Console
dimension_3 = "country"  # get country data from Search Console
dimension_4 = "device"  # Get device data from Search Console

try:
    print("Reading client_secrets.json & credentials.json..")  # Authenticate with GSC
    account = searchconsole.authenticate(
        client_config = filename,
        credentials = 'credentials.json',
    )
    webproperty = account[(url)]  # Connect to the GSC property
    print("Pulling GSC Data. Please Be Patient!")
    gsc_data = (
        webproperty.query.search_type(search_type).range(start_date, end_date)
            .dimension(
            dimension_1,
            dimension_2,
            dimension_3,
            # dimension_4,  # Dimensions can be commented out, like this one.
        )
            .get()
    )
except AttributeError:
    print("No Data Received from Search Console API!")
    sys.exit(1)
print("\nGSC Data Pull Complete!")

Reading client_secrets.json & credentials.json..
Pulling GSC Data. Please Be Patient!

GSC Data Pull Complete!


# Make The Search Console Dataframe

In [ ]:
df_gsc = pd.DataFrame(gsc_data)
df_gsc = df_gsc[df_gsc["country"].str.contains(country_filter)]  # comment this line for worldwide data
df_gsc = df_gsc.sort_values(by="clicks", ascending=False)
print(df_gsc.head(10))

                       query  \
0             external doors   
1            door superstore   
2                  shed door   
3                 shed doors   
4             interior doors   
5         wooden front doors   
6          internal doors uk   
7  internal doors with glass   
8          wooden front door   
9             internal doors   

                                                page country  clicks  \
0  https://www.doorsuperstore.co.uk/browse/extern...     gbr    2142   
1                  https://www.doorsuperstore.co.uk/     gbr    1250   
2  https://www.doorsuperstore.co.uk/browse/extern...     gbr    1145   
3  https://www.doorsuperstore.co.uk/browse/extern...     gbr    1011   
4  https://www.doorsuperstore.co.uk/browse/intern...     gbr     751   
5  https://www.doorsuperstore.co.uk/browse/extern...     gbr     557   
6  https://www.doorsuperstore.co.uk/browse/intern...     gbr     540   
7  https://www.doorsuperstore.co.uk/browse/intern...     gbr     533   

# Calculate Stats 

In [ ]:
df_gsc['no_kws_page_ranks_for'] = df_gsc['page'].map(df_gsc.groupby('page')['page'].count())
rows = len(df_gsc)
df_gsc['percent_kws_page_ranks_for'] = (df_gsc['no_kws_page_ranks_for'] / rows * 100).round(2)

df_gsc.fillna({"impressions": 0, "clicks": 0, "position": 0}, inplace=True)
df_gsc = df_gsc.astype({"impressions": int, "clicks": int, "position": int})
df_gsc['total_clicks_page'] = df_gsc['clicks'].groupby(df_gsc['page']).transform('sum')
df_gsc['total_page_imps_percent'] = df_gsc['impressions'].groupby(df_gsc['page']).transform('sum')
df_gsc['mean_ctr'] = df_gsc['ctr'].groupby(df_gsc['page']).transform('mean')
df_gsc['mean_position'] = df_gsc['position'].groupby(df_gsc['page']).transform('mean')
df_gsc['total_page_clicks_percent'] = (df_gsc['total_clicks_page'] / sum(df_gsc['clicks']) * 100).round(2)
df_gsc['page_percent_imps'] = (df_gsc['total_page_imps_percent'] / sum(df_gsc['impressions']) * 100).round(2)
df_gsc['mean_ctr'] = df_gsc['mean_ctr']* 100
df_gsc['ctr'] = df_gsc['ctr']* 100
df_gsc = df_gsc.round(2)

# Make the Stat Summary Dataframe 

In [ ]:
df_gsc_stat = df_gsc.copy()

cols = (
    "check_date",
    "page",
    "no_kws_page_ranks_for",
    "percent_kws_page_ranks_for",
    "total_clicks_page",
    "total_page_imps_percent",
    "mean_ctr",
    "mean_position",
    "total_page_clicks_percent",
    "page_percent_imps",

)

df_gsc_stat = df_gsc_stat.reindex(columns=cols)
df_gsc_stat = df_gsc_stat.sort_values(by="total_page_clicks_percent", ascending=False)
df_gsc_stat.drop_duplicates(subset=['page'], keep="first", inplace=True)
df_gsc_stat['check_date'] = str(today)
df_gsc_stat = df_gsc_stat.round(2)

df_gsc_stat["total_clicks_page"] = df_gsc_stat["total_clicks_page"].astype(int)

"""traffic ranges simplified as int so that the bar chart is generated in the 
correct order. They are renamed after the graph is generated"""

df_gsc_stat.loc[df_gsc_stat["total_clicks_page"] == 0, "traffic_range"] = 0
df_gsc_stat.loc[df_gsc_stat["total_clicks_page"].between(1, 100, inclusive=True), "traffic_range"] = 1
df_gsc_stat.loc[df_gsc_stat["total_clicks_page"].between(101, 1000, inclusive=True), "traffic_range"] = 2
df_gsc_stat.loc[df_gsc_stat["total_clicks_page"].between(1001, 5000, inclusive=True), "traffic_range"] = 3
df_gsc_stat.loc[df_gsc_stat["total_clicks_page"].between(5001, 10000, inclusive=True), "traffic_range"] = 4
df_gsc_stat.loc[df_gsc_stat["total_clicks_page"].between(10001, 1000000000, inclusive=True), "traffic_range"] = 5

df_gsc_stat.sort_values(["total_clicks_page"], ascending=False, inplace=True)
all_clicks = df_gsc_stat['total_clicks_page'].sum()
df_top_ten = df_gsc_stat.copy()
df_top_ten = df_top_ten[:10]
top_ten_clicks = df_top_ten['total_clicks_page'].sum()
percent = (top_ten_clicks / all_clicks * 100).round(2)
df_gsc_stat['top_ten_links_percentage_all_traffic'] = percent
df_gsc_stat['no_pages'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning:

Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning:

Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning:

Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: FutureWarning:

Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: FutureWarning:

Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.



# Make the Bar Chart

In [ ]:
grp = df_gsc_stat.groupby("traffic_range").agg({"total_clicks_page": "sum", "no_pages": "sum"}).reset_index()
range = grp['traffic_range']

fig = go.Figure(
    data=[
        go.Bar(name='Total Pages', x=range, y=grp['no_pages'], yaxis='y', offsetgroup=1, text = grp["no_pages"]),
        go.Bar(name='Total Clicks', x=range, y=grp['total_clicks_page'], yaxis='y2', offsetgroup=2,text = grp["total_clicks_page"])
    ],
    layout={
        'xaxis': {'title': 'Traffic Ranges'},
        'yaxis': {'title': 'Total Pages'},
        'yaxis2': {'title': 'Total Traffic', 'overlaying': 'y', 'side': 'right'}
    }
)

# Change the bar mode
fig.update_layout(barmode='group', height=800, width=1400)
fig.update_xaxes(type='category', categoryorder="category ascending")


fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 1, 2, 3, 4, 5],
        ticktext = ['No Traffic', '1-100', '101-1000', '1001-5,000', '5001-10,000', '10,001+']
    )
)

fig.update_layout(legend = dict(font = dict(family = "Arial", size = 20, color = "black")),
                  legend_title = dict(font = dict(family = "Arial", size = 20, color = "blue")))
fig.show()

# Percentage of Traffic Derived from the Top Ten Pages

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    mode = "number",
    value = df_gsc_stat['top_ten_links_percentage_all_traffic'].iloc[0],
    number = {'suffix': "%"},
    title = {"text": "What Percentage Of Clicks Do The Top Ten Pages of Your Site Generate?"},
    domain = {'x': [0, 1], 'y': [0, 1]}))
    
fig.update_layout(height=300, paper_bgcolor = "lightblue")
fig.show()

# Top Ten Pages by Traffic

In [ ]:
# rename the traffic ranges back to correct values
df_gsc_stat["traffic_range"] = df_gsc_stat["traffic_range"].astype(str)
df_gsc_stat["traffic_range"] = df_gsc_stat["traffic_range"].apply(lambda x: x.replace("0.0", "0"))
df_gsc_stat["traffic_range"] = df_gsc_stat["traffic_range"].apply(lambda x: x.replace("1.0", "1-100"))  
df_gsc_stat["traffic_range"] = df_gsc_stat["traffic_range"].apply(lambda x: x.replace("2.0", "101-1000"))  
df_gsc_stat["traffic_range"] = df_gsc_stat["traffic_range"].apply(lambda x: x.replace("3.0", "1001-5,000"))  
df_gsc_stat["traffic_range"] = df_gsc_stat["traffic_range"].apply(lambda x: x.replace("4.0", "5,001-10,000"))  
df_gsc_stat["traffic_range"] = df_gsc_stat["traffic_range"].apply(lambda x: x.replace("5.0", "10,001+"))  
top_ten = df_gsc_stat[:10]

del top_ten['no_pages']
del top_ten['top_ten_links_percentage_all_traffic']
top_ten

,check_date,page,no_kws_page_ranks_for,percent_kws_page_ranks_for,total_clicks_page,total_page_imps_percent,mean_ctr,mean_position,total_page_clicks_percent,page_percent_imps,traffic_range
0,2022-08-16,https://www.doorsuperstore.co.uk/browse/extern...,4482,3.85,7459,205587,1.57,26.10,11.80,4.80,"5,001-10,000"
1109051,2022-08-16,https://www.doorsuperstore.co.uk/browse/extern...,3143,2.70,6106,192212,1.86,12.37,9.66,4.49,"5,001-10,000"
896891,2022-08-16,https://www.doorsuperstore.co.uk/browse/intern...,3978,3.42,5985,330287,1.44,15.52,9.47,7.71,"5,001-10,000"
807081,2022-08-16,https://www.doorsuperstore.co.uk/browse/extern...,2309,1.99,5062,131767,2.12,14.22,8.01,3.08,"5,001-10,000"
799055,2022-08-16,https://www.doorsuperstore.co.uk/,2157,1.85,3275,78055,3.03,27.80,5.18,1.82,"1001-5,000"
60644,2022-08-16,https://www.doorsuperstore.co.uk/browse/intern...,2951,2.54,3085,224675,0.69,12.80,4.88,5.25,"1001-5,000"
4858,2022-08-16,https://www.doorsuperstore.co.uk/browse/intern...,3127,2.69,2355,151195,1.39,17.39,3.72,3.53,"1001-5,000"
30979,2022-08-16,https://www.doorsuperstore.co.uk/browse/intern...,1981,1.70,2251,195877,0.37,9.01,3.56,4.58,"1001-5,000"
324105,2022-08-16,https://www.doorsuperstore.co.uk/browse/intern...,1888,1.62,1557,110256,0.53,7.58,2.46,2.58,"1001-5,000"
321558,2022-08-16,https://www.doorsuperstore.co.uk/browse/door-f...,1384,1.19,1554,94101,1.96,12.38,2.46,2.20,"1001-5,000"


In [ ]:
total_rows = len(df_gsc_stat)
no_traffic = len(df_gsc_stat.loc[df_gsc_stat['traffic_range'] == "0"])
no_traffic_percent = round(no_traffic / total_rows * 100, 2)

fig = go.Figure(go.Indicator(
    mode = "number",
    value = no_traffic_percent,
    number = {'suffix': "%"},
    title = {"text": "What Percentage of Pages Have No Clicks at All?"},
    domain = {'x': [0, 1], 'y': [0, 1]}))
    
fig.update_layout(height=300, paper_bgcolor = "lightblue")

# Cleanup the Data Before Exporting



In [ ]:
zero_pages = df_gsc_stat.loc[df_gsc_stat['traffic_range'] == '0']
del zero_pages['top_ten_links_percentage_all_traffic']
del zero_pages['no_pages']
top_ten_with_keywords = pd.merge(top_ten,df_gsc, on="page", how="left")
top_ten_with_keywords['keyword_percent_traffic'] = round(top_ten_with_keywords['clicks'] / top_ten_with_keywords ['total_clicks_page_y'] * 100, 2)
top_ten_with_keywords = top_ten_with_keywords[["check_date", "page", "query", "clicks", "keyword_percent_traffic", "impressions", "ctr", "position", "traffic_range"]]
top_ten_with_keywords['all_page_traffic'] = top_ten_with_keywords['clicks'].groupby(top_ten_with_keywords['page']).transform('sum')
top_ten_with_keywords.sort_values(["all_page_traffic", "clicks"], ascending=[False, False], inplace=True,)

# Download All Pages

In [ ]:
def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0, index=False)   
    writer.save()

# list of dataframes and sheet names
dfs = [top_ten, top_ten_with_keywords, zero_pages, df_gsc_stat]
sheets = ['Top Ten Pages by Clicks','Top Ten Pages by Clicks + KWs', 'Zero Traffic Pages', 'All Pages',]    

# run function
dfs_tabs(dfs, sheets, 'top_pages.xlsx')
files.download('top_pages.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>